In [3]:
import pandas as pd
import numpy as np

# He uses 3 layers, with the hidden layers that uses 5 nodes

In [2]:
def initialize_weights(v0,v1,v2,v3,v4,w0,w1,w2,w3,w4,u0,u1,u2,u3,u4):
    return

In [9]:
def sigm_fun(x):
    return 1/(1+np.exp(-x))

sigm_fun(3)

0.9525741268224334

In [11]:
# Usa *args, come input e spacchetta

def compute_neural_network(x,v0,v1,v2,v3,v4,w0,w1,w2,w3,w4,u0,u1,u2,u3,u4):
    result = v0*sigm_fun(w0*x+u0)
    result += v1*sigm_fun(w1*x+u1)
    result += v2*sigm_fun(w2*x+u2)
    result += v3*sigm_fun(w3*x+u3)
    result += v4*sigm_fun(w4*x+u4)
    return result

In [13]:
v0,v1,v2,v3,v4,w0,w1,w2,w3,w4,u0,u1,u2,u3,u4 = 0, 0.1, 0.4, 0.5, 1, 2, 0.3, 0.5, 0.6, 0.7, 2, 1, 3, 0.3, 0.5

In [20]:
compute_neural_network(2,v0,v1,v2,v3,v4,w0,w1,w2,w3,w4,u0,u1,u2,u3,u4)

1.7546861182623799

In [26]:
# Usa *args, come input e spacchetta

def comp_neur_net(x,*weights):
    for i in weights:
        print(i)

comp_neur_net(2,*(v0,v1,v2,v3,v4,w0,w1,w2,w3,w4,u0,u1,u2,u3,u4))

0
0.1
0.4
0.5
1
2
0.3
0.5
0.6
0.7
2
1
3
0.3
0.5


In [28]:
my_vars = {}
for i in range(10):
    var_name = "var%d" % i
    my_vars[var_name] = i

print(my_vars["var2"])

# This is equivalent to creating var0 = 0, var1 = 1, and so on.

2


In [31]:
def y_n(x,*weights):
    return x*compute_neural_network(x,*weights)
y_n(2,*(v0,v1,v2,v3,v4,w0,w1,w2,w3,w4,u0,u1,u2,u3,u4))

3.5093722365247597

# Primo esempio nel paper

# 
1) Rimane da calcolare E, dove si usa la derivata in alfa

2) Dopo aver calcolato E, calcoliamo le sue derivate parziali rispetto ai pesi

3) E updatiamo i pesi